In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
from datetime import timedelta
from datetime import datetime

from airflow import DAG
from airflow.operators.python import PythonOperator


In [39]:
TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'

In [3]:
def get_data():
    top_doms = requests.get(TOP_1M_DOMAINS, stream=True)
    zipfile = ZipFile(BytesIO(top_doms.content))
    top_data = zipfile.read(TOP_1M_DOMAINS_FILE).decode('utf-8')

    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)


In [ ]:
top_data_df

In [40]:
# Найти топ-10 доменных зон по численности доменов

# TOP_1M_DOMAINS_FILE поменяла на TOP_1M_DOMAINS, чтобы работало здесь
top_data_df = pd.read_csv(TOP_1M_DOMAINS, names=['rank', 'domain'])
top_data_df['domain_zone'] = top_data_df['domain'].str.split('.').str[-1]
top_data_df.groupby('Domain_zone', as_index = False)\
            .agg({'domain':'count'})\
            .sort_values('domain', ascending = False).head(10)

,Domain_zone,domain
150,com,404402
498,org,34803
470,net,29464
569,ru,20909
180,de,15586
34,au,14076
481,nl,13715
688,uk,12765
357,it,12078
523,pl,10186


In [27]:
top_data_df

,rank,domain,Domain_zone
0,1,google.com,com
1,2,youtube.com,com
2,3,facebook.com,com
3,4,baidu.com,com
4,5,bilibili.com,com
...,...,...,...
821337,821338,zvgrafosradio.blogspot.com,com
821338,821339,zvideos.com,com
821339,821340,zylax.com.au,au
821340,821341,zyrontech.com.au,au


In [ ]:
# Найти домен с самым длинным именем (если их несколько, то взять только первый в алфавитном порядке)

In [28]:
top_data_df['lenth'] = top_data_df['domain'].str.len()

In [58]:
max_lenth = top_data_df.sort_values(['lenth', 'domain'], ascending = (False, True)).head(1)

In [55]:
max_lenth.reset_index()

,index,domain
0,169809,file-service-default-114c67af0763a8a98e770ff3e...


In [59]:
max_lenth['domain']

169809    file-service-default-114c67af0763a8a98e770ff3e...
Name: domain, dtype: object

In [ ]:
max_lenth

In [ ]:
# На каком месте находится домен airflow.com?

In [41]:
top_data_df['lenth'] = top_data_df['domain'].str.len()

In [42]:
top_lenth_df = top_data_df.sort_values(['lenth', 'domain'], ascending = (False, True))

In [49]:
top_lenth_df = top_lenth_df.reset_index()

In [50]:
top_lenth_df.reset_index()

,level_0,index,rank,domain,Domain_zone,lenth
0,0,169809,169810,file-service-default-114c67af0763a8a98e770ff3e...,com,81
1,1,368205,368206,aaalert-cal-nsow-xx-ssmmce-ccdc-cdcdc-cklmll-d...,net,78
2,2,551802,551803,alert-callnow-fo-supoer-nx-wnpz-xdsx-xsx--skkk...,net,78
3,3,610502,610503,ssaalert-cal-now-xx-ssm-mce-cdc-cdcdc-cklmll-d...,net,78
4,4,189791,189792,aaalert-cal-nosw-xx-ssmmce-ccdc-cdcdc-cklmll-d...,net,77
...,...,...,...,...,...,...
821337,821337,753261,753262,w.ai,ai,4
821338,821338,76032,76033,w.tt,tt,4
821339,821339,35286,35287,x.co,co,4
821340,821340,443538,443539,x.gd,gd,4


In [44]:
top_lenth_df.query('domain ==  "airflow.com"')

,index,rank,domain,Domain_zone,lenth


NameError: name 'google' is not defined

In [ ]:
def get_stat():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_data_df['Domain_zone'] = top_data_df['domain'].str.split('.').str[-1]
    top_data_top_10 = top_data_top_10.head(10)
    with open('top_data_top_10.csv', 'w') as f:
        f.write(top_data_top_10.to_csv(index=False, header=False))

In [ ]:

def get_stat_com():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_data_top_10 = top_data_df[top_data_df['domain'].str.endswith('.com')]
    top_data_top_10 = top_data_top_10.head(10)
    with open('top_data_top_10_com.csv', 'w') as f:
        f.write(top_data_top_10.to_csv(index=False, header=False))

In [ ]:

def print_data(ds): # передаем глобальную переменную airflow
    with open('top_data_top_10.csv', 'r') as f:
        all_data = f.read()
    with open('top_data_top_10_com.csv', 'r') as f:
        all_data_com = f.read()
    date = ds

    print(f'Top domains in .RU for date {date}')
    print(all_data)

    print(f'Top domains in .COM for date {date}')
    print(all_data_com)

In [ ]:
default_args = {
    'owner': 'a.batalov',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2021, 10, 7),
    'schedule_interval': '0 12 * * *'
}
dag = DAG('top_10_ru_new', default_args=default_args)

In [ ]:
t1 = PythonOperator(task_id='get_data',
                    python_callable=get_data,
                    dag=dag)

t2 = PythonOperator(task_id='get_stat',
                    python_callable=get_stat,
                    dag=dag)

t2_com = PythonOperator(task_id='get_stat_com',
                        python_callable=get_stat_com,
                        dag=dag)

t3 = PythonOperator(task_id='print_data',
                    python_callable=print_data,
                    dag=dag)

In [ ]:
t1 >> [t2, t2_com] >> t3

In [8]:
TOP_1M_DOMAINS_FILE

'top-1m.csv'

In [60]:
top_data_df['domain_name'] = top_data_df['domain'].str.split('.').str[-2]
domain_place=top_data_df.query('domain_name=="airflow"')
    if domain_place.empty:
        print('not_found')
    else:
        print(top_data_df.rank)

IndentationError: unexpected indent (<ipython-input-60-8c01e5d7d627>, line 3)

In [63]:
if top_data_df[top_data_df['domain']== 'airflow.com'].empty:
      f.write('airflow.com is not found')
else: 
      rank = top_data_df[top_data_df['domain']== 'airflow.com']['rank']

NameError: name 'f' is not defined

In [64]:
if top_data_df[top_data_df['domain']== 'airflow.com'].empty:
      print('airflow.com is not found')
else: 
       rank = top_data_df[top_data_df['domain']== 'airflow.com']['rank']

airflow.com is not found
